In [1]:
import pandas as pd

In [90]:
access_path = '../../data/processed/'
# bal_file = 'bal_btw_rebal_20210702.dat'
bal_file = 'balance_rebaldate_0702_all.dat'

#### 매일의 잔고(detail 테이블) INSERT

In [91]:
df_bal = pd.read_csv(access_path+bal_file, parse_dates=['date'])

In [92]:
df_bal.date = df_bal['date'].dt.strftime('%#m/%#d/%Y 4:%M:%S PM')

In [93]:
df_bal

,date,itemcode,userid,name,asset_class,itemname,quantity,cost_price,cost_value,price,value,wt,group_by,original
0,7/2/2021 4:00:00 PM,A114260,A001,안정추구형소규모,Fixed Income,KODEX 국고채3년,2.0,57845.0,115690.0,57845.0,115690.0,0.162228,NaN,Rebal
1,7/2/2021 4:00:00 PM,A122260,A001,안정추구형소규모,Fixed Income,KOSEF 통안채1년,1.0,101120.0,101120.0,101120.0,101120.0,0.141797,NaN,Rebal
2,7/2/2021 4:00:00 PM,A130730,A001,안정추구형소규모,Fixed Income,KOSEF 단기자금,2.0,101060.0,202120.0,101060.0,202120.0,0.283427,NaN,Rebal
3,7/2/2021 4:00:00 PM,A196230,A001,안정추구형소규모,Fixed Income,KBSTAR 단기통안채,1.0,104430.0,104430.0,104430.0,104430.0,0.146439,NaN,Rebal
4,7/2/2021 4:00:00 PM,A273130,A001,안정추구형소규모,Fixed Income,KODEX 종합채권(AA-이상)액티브,1.0,108635.0,108635.0,108635.0,108635.0,0.152335,NaN,Rebal
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
67,7/2/2021 4:00:00 PM,A157490,A009,적극투자형대규모,Equity,TIGER 소프트웨어,17.0,19565.0,332605.0,19565.0,332605.0,0.243277,NaN,Rebal
68,7/2/2021 4:00:00 PM,A266370,A009,적극투자형대규모,Equity,KODEX IT,9.0,21175.0,190575.0,21175.0,190575.0,0.139392,NaN,Rebal
69,7/2/2021 4:00:00 PM,A069500,A009,적극투자형대규모,Equity,KODEX 200,3.0,43640.0,130920.0,43640.0,130920.0,0.095759,NaN,Rebal
70,7/2/2021 4:00:00 PM,A114260,A009,적극투자형대규모,Fixed Income,KODEX 국고채3년,1.0,57845.0,57845.0,57845.0,57845.0,0.042310,NaN,Rebal


In [94]:
from sqlalchemy import create_engine

# Defining our connection variables
username = 'kdyjpzwsyqrqpb' # replace with your username
password = '6b610da9417e361c2af3be1fd0c179061b5b258f0e6f7ff9e1da464ba221e46c' # replace with your password
ipaddress = 'ec2-35-174-35-242.compute-1.amazonaws.com' # change this to your db’s IP address is if not Panoply
port = 5432 # this is the standard port for Postgres, but change it to your port if needed
dbname = 'd2ke5vkurjkusr' # change this to...you guessed it...the name of your db

# A long string that contains the necessary Postgres login information
postgres_str = f'postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
          
# Create the connection
cnx = create_engine(postgres_str)

In [95]:
table_nm = 'detail'
df_bal.to_sql(table_nm, con=cnx, index=False, if_exists='append')

### 자산군별 비중(general 테이블) INSERT

In [96]:
df_bal = pd.read_csv(access_path+bal_file, parse_dates=['date'])

In [97]:
df_bal_gby = df_bal.loc[:, ['date', 'userid', 'value', 'asset_class']].sort_values(
    ['date', 'userid'],
    ascending=True).groupby(['date', 'userid', 'asset_class'
                             ]).sum().reset_index(drop=False)

In [98]:
df_bal_gby = df_bal_gby.assign(wt=df_bal_gby.groupby(['date', 'userid'])['value'].transform(lambda x: x/x.sum()))

In [99]:
df_bal_gby.date = df_bal_gby['date'].dt.strftime('%#m/%#d/%Y 4:%M:%S PM')

In [100]:
df_bal_gby

,date,userid,asset_class,value,wt
0,7/2/2021 4:00:00 PM,A001,Alternative,12085.0,0.016946
1,7/2/2021 4:00:00 PM,A001,Cash,6210.0,0.008708
2,7/2/2021 4:00:00 PM,A001,Equity,62840.0,0.088119
3,7/2/2021 4:00:00 PM,A001,Fixed Income,631995.0,0.886227
4,7/2/2021 4:00:00 PM,A002,Alternative,24170.0,0.024097
5,7/2/2021 4:00:00 PM,A002,Cash,4355.0,0.004342
6,7/2/2021 4:00:00 PM,A002,Equity,125680.0,0.125303
7,7/2/2021 4:00:00 PM,A002,Fixed Income,848805.0,0.846258
8,7/2/2021 4:00:00 PM,A003,Alternative,36255.0,0.027793
9,7/2/2021 4:00:00 PM,A003,Cash,14500.0,0.011116


In [101]:
table_nm = 'general'
df_bal_gby.to_sql(table_nm, con=cnx, index=False, if_exists='append')